In [1]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
from torch.autograd import Variable
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim as optim
from pathlib import Path
print("importé")

importé


In [69]:
#Lecture des fichiers EGC

#Reading and labeling data 
#Ne pas prendre en considération les 10s au début et à la fin : pause 
hz=200
entries = Path('/home/asma/Desktop/data')
#print(entries)

#Liste des  fichiers dans le répertoire 

for entry in entries.iterdir():
    
    
    path_file = entries / entry.name
    
    #Not test files
    #print("entry.name",entry.name)
    if "test" not in entry.name and "mix" not in entry.name  : 
        
        
        
        table1 = np.empty([640,800,4])
        
        j = 0 #Initialization
        
        for path in path_file.iterdir():
            
            print("path",path)
            #Read files
            
           
            with open(path) as f:
               
                flat_list = f.readlines()
                
                #Remove header lines and A0 pause seconds
                start = 6+10*hz
                end = start + 4*hz
               
                
                # Loop to read all the activities
                while(end<14806):  
                    j = j+1
                    activity=[]
                    table = [] #Table Initialisation
                    #read the lines of all the time points 200*4  of "One activity"  
                    for m in range(hz*4):
                       
                        table = np.asarray(flat_list[start:end], dtype=np.unicode)  
                    
                    #Remove "," and extract egg values of one activity
                    for i in range (table.shape[0]):
                            line = table[i].split(',') #eliminate ","
                            activity.append(line[1:5]) # extract 4 channels egg
                            
                    
                    
                    
                   
                    print(j)
                    table1[j,0:800,:] = np.asarray(activity, dtype=np.float64) 
                    
            
            #print(table1[j,0:800,:])

                  
                    
                    
                    
                    
                    
                   
                            
                        
                        #line =[ line for line in flat_list[start:end]]
                        
                       # print(line[0])
                       
                    #table = np.asarray(line, dtype=np.float64)  
                    #print(table.shape)
                       
                    
      
             
                    
#If right,left, up, down center


path /home/asma/Desktop/data/up/Test_U_4_5.txt
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
2

IndexError: index 640 is out of bounds for axis 0 with size 640

In [ ]:
      
                        end = end + 4*hz
                        
                        
                        
                        #eeg = [word for word in line[0:4]]
                        eeg = []
                        repetition = 0
                        for word in line :
                            if (repetition<4):
                                eeg.append(word)
                                repetition = repetition +1 
                            
                          
                        #print("eeg",eeg)
                        
                        #table[k:j:] = [np.asarray(word, dtype=np.float64) for word in line[0:4]]
                        table.append(np.asarray(eeg, dtype=np.float64))#Extract the 4 channels values
                        
                        
                        
                        j=j+1
                
                
                k = k+1 
        
        #print(table[1:1:])
        print("table",len(table))
        print("k,j",k,j)
        

In [3]:
class Model(torch.nn.Module):
    # Defines all the layers of the model
    def __init__(self, num_channels, num_classes, sample_length):
        super(Model, self).__init__()
        self.sample_length = sample_length
    
        self.block1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, (1, num_channels), padding = 0),
            torch.nn.ELU(),
            torch.nn.BatchNorm2d(16, False),
            torch.nn.Dropout(0.5)
            )
        
        self.block2 = torch.nn.Sequential(
            torch.nn.ZeroPad2d((16, 17, 0, 1)), 
            torch.nn.Conv2d(1, 4, (2, 32)),
            torch.nn.ELU(),
            torch.nn.BatchNorm2d(4, False),
            torch.nn.Dropout(0.25),
            torch.nn.MaxPool2d((2, 4))
            )
        
        self.block3 = torch.nn.Sequential(
            torch.nn.ZeroPad2d((2, 1, 4, 3)),
            torch.nn.Conv2d(4, 4, (8, 4)),
            torch.nn.ELU(),
            torch.nn.BatchNorm2d(4, False),
            torch.nn.Dropout(0.25),
            torch.nn.MaxPool2d((2, 4))
            )
        
        # Dense/fully connected layer
        # FC Layer\n",
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.\n",
        # I have 120 timepoints. \n",
        #self.fc1 = nn.Linear(4*2*7, 1)
        
        '''
        x avant torch.Size([25, 4, 4, 7])
        
        self.fc = torch.nn.Linear(4*4*7*sample_length, num_classes)
        
        x apres torch.Size([25, 112])
        output après torch.Size([25, 5])
        '''
        # secondes: x avant torch.Size([25, 4, 4, 30]) (4*4*30=480)
        
        self.fc = torch.nn.Linear(4*1*30*sample_length, num_classes)
        
        '''
        x apres torch.Size([10, 1200])
        output après torch.Size([10, 5])
        '''
        
    def forward(self, x):
        x = self.block1(x).permute(0, 3, 1, 2)
        x = self.block2(x)
        x = self.block3(x)
        
        
        print("x avant",x.shape)
        x = x.view(-1, 4*1*30*self.sample_length)
        #print("self.sample_length",self.sample_length)
        print("x apres",x.shape)
        
        x = self.fc(x)
        print("output après",x.shape)
        return x

#def __init__(self, num_channels, num_classes, sample_length):
#net = Model(channels, len(subjects), sample_length).to(device)
'''
hz = 128 # Datapoints per second per channel
channels = 14 # Number of channels
sample_length = 30 # Number of seconds per sample
model = Model(channels, len(subjects), sample_length).to(device)

'''

channels=4
len_subjects=5 #Number of classes
hz=120
sample_length=4 # Number of seconds per sample
learning_rate = 0.001

net = Model(channels, len_subjects, sample_length).cuda(0)


#print( net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 64)).cuda(0)))
criterion = torch.nn.CrossEntropyLoss() # Loss function (build-in softmax)
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate) # Optimizer algorithm

#### Evaluate function returns values of different criteria like accuracy, precision etc. 
In case you face memory overflow issues, use batch size to control how many samples get evaluated at one time. Use a batch_size that is a factor of length of samples. This ensures that you won't miss any samples.

In [ ]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 100
    
    predicted = []
    
    for i in range (int(len(X)/batch_size)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]).cuda(0))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    inputs = Variable(torch.from_numpy(X).cuda(0))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    print("Y.shape",Y)
    print("predicted",predicted)
    
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted), normalize=False))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results

#### Generate random data

##### Data format:
Datatype - float32 (both X and Y) <br>
X.shape - (#samples, 1, #timepoints,  #channels) <br>
Y.shape - (#samples)

In [ ]:
pip install torchtext==0.2.3

In [4]:
hz = 120
X_train = np.random.rand(100, 1, hz*sample_length, 4).astype('float32') # np.random.rand generates between [0, 1)
y_train = np.round(np.random.rand(100).astype('float32')) # binary data, so we round it to 0 or 1.

X_val = np.random.rand(100, 1, hz*sample_length, 4).astype('float32')
y_val = np.round(np.random.rand(100).astype('float32'))

X_test = np.random.rand(100, 1, hz*sample_length, 4).astype('float32')
y_test = np.round(np.random.rand(100).astype('float32'))
print(X_train.shape)

print(y_test)
print(X_train.shape)

(100, 1, 480, 4)
[1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1.
 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0.
 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
 0. 0. 1. 0.]
(100, 1, 480, 4)


#### Run

In [ ]:
def accuracy(model, device, batch_size, x_data, y_data):
    net.eval()
    
    corrects = 0
    #with torch.no_grad():
    end = 0
    for i in range((len(x_data) // batch_size)):
        start = i * batch_size
        end = start + batch_size

        x = Variable(torch.Tensor(x_data[start:end]).cuda(0))
        #y = torch.Tensor(y_data[start:end], dtype=torch.long, device=device)
        y = torch.Tensor(y_data[start:end]).long().cuda(0)

        output = model(x)
        args = torch.max(output.data, dim=1)[1]
       
        
        corrects += torch.sum(args == y)

    if end < (len(x_data) - 1):
        x = torch.tensor(x_data[end:], dtype=torch.float32, device=device)
        y = torch.tensor(y_data[end:], dtype=torch.long, device=device)

        output = model(x)
        args = torch.max(output.data, dim=1)[1]

        corrects += torch.sum(args == y).item()             
    
    return corrects/len(x_data)

In [ ]:
batch_size = 25

device = torch.cuda.set_device(0)
test_acc = accuracy(net, device, batch_size, X_test, y_test)*100
train_acc = accuracy(net, device, batch_size, X_train, y_train)*100

print("Epoch: 000 | Test accuracy: {0:0.3f}% | Train accuracy: {1:0.3f}%".format(test_acc, train_acc))

for epoch in range(10):  # loop over the dataset multiple times
    print ("\nEpoch ", epoch)
    
    
    
    running_loss = 0.0
    print(len(X_train))
    
    for i in range(int(len(X_train)/batch_size)):
        print ("\nbatch ", i)
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X_train[s:e])
        
        print("inputs.shape",inputs.shape)
        
        
        labels = torch.FloatTensor(np.array([y_train[s:e]]).T*1.0).long()
        
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        #print("input",inputs.shape)
        '''
        output = model(x)
        # Calculates lost
        loss = criterion(output, y)
        
        # Optimizes the model using backpropagation and the optimizer algorithm 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
        '''
        
        outputs = net(inputs)
        
        #print(outputs)
        
        #target doit etre long
        
       
        
        criterion = nn.CrossEntropyLoss()
        #print("labels",labels.shape)
        #labels = labels.squeeze(1)
        #print("labels",labels.shape)
        
        #print("outputs shape",outputs.shape)
        
        loss = criterion(outputs,labels.squeeze(1))
        loss.backward()
        
        
        optimizer.step()
        
        running_loss += loss.data[0]
        
    
    
    test_acc = accuracy(net, device, batch_size, X_test, y_test)*100
    train_acc = accuracy(net, device, batch_size, X_train, y_train)*100

    print("Epoch: {0:03d} | Test accuracy: {0:0.3f}% | Train accuracy: {1:0.3f}%".format(epoch+1, test_acc, train_acc))
    # Validation accuracy
    '''
    params = ["acc", "auc", "fmeasure"]
    print (params)
    print ("Training Loss ", running_loss)
   
    print ("Train - ", evaluate(net, X_train, y_train.round(), params))
    print ("Validation - ", evaluate(net, X_val, y_val, params))
    print ("Test - ", evaluate(net, X_test, y_test, params))
    '''
    
    